# Determine the offset values

Using the frames where the plasma is expected to be at the centroid, the relationship between the offset value in each probe array and plasma current is determined

## approximate frames at centroid

In [1]:
import cv2
import shutil
from matplotlib import pyplot as plt
from skimage.metrics import structural_similarity
from tqdm import tqdm

retreive_img_directory = lambda shot_no: f"resources\\TTI frame\\{shot_no}\\{shot_no}_frames_jpg\\"
centroid_directory = r"resources\\centroidFrames"

img_directory =  retreive_img_directory(966)

all_shot_no = [370,433,582,665,920,926,928] + list(range(960,971)) + [1108,1275,1745,1804,2308]
base_frame_dict = {
    433:187, 582:160, 665:189, 926:160, 960:155, 961:192, 962:199, 963:155, 964: 154, 965: 200, 966:200, 967:208, 968:154,969:205,
    970:209,1108:184,1275:184,1745:181,1804:189,2308:198
}

In [ ]:
similarity = []
for shot_no in all_shot_no:
    if shot_no in [370,920,928]:continue

    img_directory = retreive_img_directory(shot_no)
    for i in [-1,0,1]:
        frame = base_frame_dict[shot_no] + i
        img_path = img_directory + f"{frame}.jpg"
        shutil.copy(img_path,centroid_directory + f"\\{shot_no}_{frame}.jpg")

## Calculate offset for each frame

In [20]:
from toroidalFilament_dir.geometry_TT1 import all_arrays, coil_angle_dict, probe_lst_to_str
from toroidalFilament_dir.DxDz import cal_newton_DxDz as cal_DxDz
from toroidalFilament_dir.signal_strength import cal_signal

import numpy as np
import os
import pandas as pd

Ip_df = pd.read_excel(r"C:\Users\pitit\Documents\02_MUIC_programming\ICPY_441_Senior_project_in_physics\plasmaColumnPosition\resources\magneticSignal\Plasma current for plasma position.xlsx",sheet_name="Sheet1")

In [6]:
x_shift, z_shift = 0,0

center_Dx = []
for probe_arr in all_arrays:
    probe_angles = [coil_angle_dict[probe] for probe in probe_arr]
    signal = cal_signal(x_shift,z_shift,probe_angles)
    Dx, Dz = cal_DxDz(signal,probe_angles)
    center_Dx.append(Dx)

mean_Dx_offset = np.mean(center_Dx)

print("mean value of Dx at centroid is" + f" {mean_Dx_offset} m")

mean value of Dx at centroid is -0.204920003280666 m


In [ ]:
def find_closest(arr, target): #use binary search to find element with closest value to the target
    res = arr[0]
    lo = 0
    hi = len(arr) - 1

    while lo <= hi:
        mid = lo + (hi - lo) // 2

        # Update res if mid is closer to target
        if abs(arr[mid] - target) < abs(res - target):
            res = arr[mid]

        # In case of a tie, prefer larger value
        elif abs(arr[mid] - target) == abs(res - target):
            res = max(res, arr[mid])

        if arr[mid] == target:
            return arr[mid]
        elif arr[mid] < target:
            lo = mid + 1
        else:
            hi = mid - 1

    return res

149.994

In [ ]:
keys = [probe_lst_to_str(probe_arr) for probe_arr in all_arrays]
offset_dict = {
    #index 0 -> plasma current
    #index 1 -> Dx offset
    probe_arr : [[],[]] for probe_arr in keys 
}

def time_from_frame(frame):
    time = frame/2 + 260 
    return find_closest(Ip_df["Time [ms]"],time)

def find_Ip(shot_no,time):
    row = Ip_df.loc[Ip_df["Time [ms]"] == time].index[0]
    column = Ip_df.columns.get_loc(shot_no)
    return Ip_df.iloc[row,column]

def find_signal(shot_no,time,probe_arr):
    return 

def cal_offset(Dx,Dx0 = mean_Dx_offset):
    return Dx - Dx0

shot_no_from_path = lambda path: path[0:4]
frame_from_path = lambda path: path[-7:-4]

for img_path in os.listdir(r"resources\centroidFrames"):
    #determine shot number and frame of the frame to be analyzed
    shot_no = int(shot_no_from_path(img_path))
    frame = int(frame_from_path(img_path))

    time = time_from_frame(frame)

    #determine plasma current of given shot and frame
    Ip = find_Ip(shot_no,time)

    for probe_arr in all_arrays:
        #retreive signal at time
        signal = find_signal(shot_no, time,probe_arr)

        #calculate original Dx and Dz
        Dx, Dz = cal_DxDz(signal,c_angle = [coil_angle_dict[probe_num] for probe_num in probe_arr])

        #calculate offset
        offset_Dx = cal_offset(Dx)

        probe_key = probe_lst_to_str(probe_arr)
        offset_dict[probe_key][0].append(Ip)
        offset_dict[probe_key][1].append(offset_Dx)

61620.425781


TypeError: 'NoneType' object is not subscriptable